# TF-IDF: Term Frequency–Inverse Document Frequency

TF-IDF is a numerical statistic that is intended to reflect how important 
a word is to a document in a collection or corpus. It is often used as a 
weighting factor in searches of information retrieval, 
text mining, and user modeling. The tf–idf value increases proportionally to 
the number of times a word appears in the document and 
is offset by the number of documents in the corpus 
that contain the word, which helps to adjust for the fact that some words 
appear more frequently in general. TF-IDF is one of 
the most popular term-weighting schemes today. A survey conducted 
in 2015 showed that 83% of text-based recommender systems in digital 
libraries use tf–idf.

In this post, we will:

- learn how to build up a tf-idf model from scratch
- apply it to match names (entities) 

TF-IDF was invented by [Karen Jones](https://en.wikipedia.org/wiki/Karen_Sp%C3%A4rck_Jones),
her original papers are among the most cited papers in the field of CIS.

## Definition

- TF: term frequency is the number of times a token appears in a document / number of words in document
- IDF: inverse document frequency is a weight indicating how commonly a 
word is used. The more frequency its usage across documents, the lower its score.
For instance, 'the' shows up very often, it has a low score of IDF, The lower
the score, the less important the world becomes.

The formula to compute the tf-idf for a token $t$ of a document $d$
in a document set is:

$$tf-idf = tf(t, d) \times idf(t), \quad idf(t) = \log(\frac{n}{df(t)+1})$$

where $n$ is the total number of documents in the document sett and 
$df(t)$ is the _document frequency_ of $t$; the document frequency is
the number of documents in the document set that contains the term $t$.


For the TF, here is the formula:

$$tf(t, d) = \frac{\text{count}_{t, d}}{\text{len}(d)}, \quad d \in [d_1, d_2, \cdots, d_n]$$

For the IDF, we will implement the smooth version based on the document of 
`sklearn`, which is 

$$idf(t) = \log \left ( \frac{1+n}{1+df(t)} \right ) +1 $$

In [1]:
import time
import struct
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mplt
import pandas as pd
from collections import Counter
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
%config InlineBackend.figure_formats = ['svg']

<div>
<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
  "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<svg xmlns:xlink="http://www.w3.org/1999/xlink" width="405pt" height="188.949375pt" viewBox="0 0 405 188.949375" xmlns="http://www.w3.org/2000/svg" version="1.1">
 <metadata>
  <rdf:RDF xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:cc="http://creativecommons.org/ns#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
   <cc:Work>
    <dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
    <dc:date>2023-01-15T00:26:11.286127</dc:date>
    <dc:format>image/svg+xml</dc:format>
    <dc:creator>
     <cc:Agent>
      <dc:title>Matplotlib v3.5.2, https://matplotlib.org/</dc:title>
     </cc:Agent>
    </dc:creator>
   </cc:Work>
  </rdf:RDF>
 </metadata>
 <defs>
  <style type="text/css">*{stroke-linejoin: round; stroke-linecap: butt}</style>
 </defs>
 <g id="figure_1">
  <g id="patch_1">
   <path d="M 0 188.949375 
L 405 188.949375 
L 405 0 
L 0 0 
L 0 188.949375 
z
" style="fill: none"/>
  </g>
  <g id="axes_1">
   <g clip-path="url(#p45ab502d97)">
    <image xlink:href="data:image/png;base64,
..." id="image0309c1b816" transform="scale(1 -1)translate(0 -174.72)" x="7.2" y="-7.029375" width="390.72" height="174.72"/>
   </g>
  </g>
 </g>
 <defs>
  <clipPath id="p45ab502d97">
   <rect x="7.2" y="7.2" width="390.6" height="174.549375"/>
  </clipPath>
 </defs>
</svg>

</div>

In [2]:
# document example a list
corpus_1 = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]
corpus_1 = np.array(corpus_1)
corpus_1

array(['this is the first document',
       'this document is the second document',
       'and this is the third one', 'is this the first document'],
      dtype='<U36')

In [7]:
def construct_dimension(corpus: np.ndarray) -> dict:
     """
     Create a basis of token based on unique tokens in the corpus,
     which could be used in the matrix of feature extractions
     
     Parameters
     ----------
     corpus: np.ndarray, shape (n, ) or (n, 1), where each row is a 
     a long string with spaces such as "this is the first document"
     
     Returns
     ----------
     unique_tokens_dict: dict{'token': dimension: int}
     """
     corpus = corpus.ravel()  # make it shape (n, )
     
     unique_tokens = []
     
     # loop over rows of corpus np.ndarray
     for row in corpus:
          # split token 
          # you might need different loop based on different input
          # here we are dealing with a long string with spaces
          # your row input might be array or list of token 
          for token in row.split():  # split based on space 
               # ignore alphabet as len('b') = 1
               if len(token) >= 2 and token not in unique_tokens:
                    unique_tokens.append(token)
          # sort it 
          unique_tokens.sort()
          
          unique_tokens_dict = {token:i for i, token in enumerate(unique_tokens)}
          
     return unique_tokens_dict

In [5]:
construct_dimension(corpus_1)

{'and': 0,
 'document': 1,
 'first': 2,
 'is': 3,
 'one': 4,
 'second': 5,
 'the': 6,
 'third': 7,
 'this': 8}

In [9]:
def count_word(corpus: np.ndarray, word: str) -> int:
    """
    Count word frequency from the corpus
    
    Parameters
    -----------
    corpus: np.ndarray, shape (n, ) or (n, 1)
    word: string
    
    Return
    ------------
    count: integer, word frequency
    """
    corpus = corpus.ravel()
    count = 0
    
    for row in corpus:
        # assumes row is string 
        if word in row:
            count += 1
    
    return count 

The two functions above are very easy to understand. We will use them to 
calculate the term frequency. Before we proceed, let's recap:

- we have documents
- we have texts in each document 
- we have a set of unique words in the texts of each document
- we have a set of unique words in all the documents 

To calculate the inverse document frequency (IDF), we will use the following
formula:

$$idf(t) = \log \left ( \frac{1+n}{1+df(t)} \right ) +1 $$

- $n$ is the total number of documents
- $df(t)$ is the number of documents containing term $t$

Since we have _terms_ and _documents_ we will construct a __matrix__ 
to trace the different kinds of frequencies. 

<div>
<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
  "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<svg xmlns:xlink="http://www.w3.org/1999/xlink" width="405pt" height="309.38112pt" viewBox="0 0 405 309.38112" xmlns="http://www.w3.org/2000/svg" version="1.1">
 <metadata>
  <rdf:RDF xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:cc="http://creativecommons.org/ns#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
   <cc:Work>
    <dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
    <dc:date>2023-01-15T00:31:19.524873</dc:date>
    <dc:format>image/svg+xml</dc:format>
    <dc:creator>
     <cc:Agent>
      <dc:title>Matplotlib v3.5.2, https://matplotlib.org/</dc:title>
     </cc:Agent>
    </dc:creator>
   </cc:Work>
  </rdf:RDF>
 </metadata>
 <defs>
  <style type="text/css">*{stroke-linejoin: round; stroke-linecap: butt}</style>
 </defs>
 <g id="figure_1">
  <g id="patch_1">
   <path d="M 0 309.38112 
L 405 309.38112 
L 405 0 
L 0 0 
L 0 309.38112 
z
" style="fill: none"/>
  </g>
  <g id="axes_1">
   <g clip-path="url(#pf202efc34c)">
    <image xlink:href="data:image/png;base64,
..." id="image5cdf28b6e2" transform="scale(1 -1)translate(0 -295.2)" x="7.2" y="-6.98112" width="390.72" height="295.2"/>
   </g>
  </g>
 </g>
 <defs>
  <clipPath id="pf202efc34c">
   <rect x="7.2" y="7.2" width="390.6" height="294.98112"/>
  </clipPath>
 </defs>
</svg>

</div>

## Compressed sparse row (CSR) format

Since the above matrix with rows as documents and columns as terms are very
sparse. If you check the illustration again, you will notice that we have
many zeros in that matrix.

To store a sparse matrix, we will use __compressed sparse row (CSR)__ format.
Comparing with other formats to store a sparse matrix, such as Dictionary of Key (DOK), 
List of lists (LIL), and Coordinate list (COO), and Compressed sparse column (CSC),
here are its advantages and disadvantages:

- Advantages of the CSR format
    - efficient arithmetic operations CSR + CSR, CSR * CSR, etc.
    - efficient row slicing
    - fast matrix vector products

- Disadvantages of the CSR format
    -slow column slicing operations (consider CSC)
    - changes to the sparsity structure are expensive (consider LIL or DOK)

<div>
<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
  "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<svg xmlns:xlink="http://www.w3.org/1999/xlink" width="405pt" height="137.335622pt" viewBox="0 0 405 137.335622" xmlns="http://www.w3.org/2000/svg" version="1.1">
 <metadata>
  <rdf:RDF xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:cc="http://creativecommons.org/ns#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
   <cc:Work>
    <dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
    <dc:date>2023-01-15T11:34:17.424213</dc:date>
    <dc:format>image/svg+xml</dc:format>
    <dc:creator>
     <cc:Agent>
      <dc:title>Matplotlib v3.5.2, https://matplotlib.org/</dc:title>
     </cc:Agent>
    </dc:creator>
   </cc:Work>
  </rdf:RDF>
 </metadata>
 <defs>
  <style type="text/css">*{stroke-linejoin: round; stroke-linecap: butt}</style>
 </defs>
 <g id="figure_1">
  <g id="patch_1">
   <path d="M 0 137.335622 
L 405 137.335622 
L 405 0 
L 0 0 
L 0 137.335622 
z
" style="fill: none"/>
  </g>
  <g id="axes_1">
   <g clip-path="url(#p4bdeca3972)">
    <image xlink:href="data:image/png;base64,
..." id="imaged3467d4614" transform="scale(1 -1)translate(0 -123.12)" x="7.2" y="-7.015622" width="390.72" height="123.12"/>
   </g>
  </g>
 </g>
 <defs>
  <clipPath id="p4bdeca3972">
   <rect x="7.2" y="7.2" width="390.6" height="122.935622"/>
  </clipPath>
 </defs>
</svg>

</div>

n CSR, a matrix is stored as three one-dimensional arrays of row pointers, 
column indices and values, where the first two are of integer type and the 
last one of float type, usually double (See the above illustration, __color__ matters). 

Suppose we have a matrix of $m \times n$.

As the name suggests, non-zero entries are stored per row, which means we 
pick up non-zero elements in row-wise fashion. For example, we have $10, -2$
in the first row of our example. Those elements will be stored in an array
called _values array_. 

where each non-zero is defined by a pair of column index and corresponding value. 
For instance, the column index of $10, -2$ are $0, 4$, which are stored in 
the _column indices array_ (see the below color). 

_The column indices and values arrays therefore have a length equal to the total 
number of non-zero entries_. 

Row indices are given implicitly by the row pointer array, which contains 
_the starting index in the column index and values arrays_ for the non-zero 
entries of each row. 

The _row pointer array_ has length of $m+1$. Instead of point to the row index 
of matrix $A$, we will point to the _column indices array_. For instance,
the green element of row pointer array is $2$, which indicates that the elements
of second row starts at $\text{column\_indices\_array}[2]$. 

For the row pointer array $\text{row\_pointer\_array}[i+1] - \text{row\_pointer\_array}[i]$
also gives _how many elements for $A[i, :]$.  

To restore matrix $A$ from the CSR format, we have the following steps:

- step 1: get the row idx $i$ from loop over `length(row_pointer_array)-1`
- step 2: get the column idx $j$ for each row from _column indices array_ with a nested loop
- step 3: get the value from values array with a nested loop (after step 1)
- step 4: assign the value into $A[i, j]$

_Remark_: we did not talk about the length of columns, therefore one needs 
to know the value of $n$ if she/he wants to restore matrix $A$ from
CSR formats.

In [10]:
# the same example
row_ptr = np.array([0, 2, 4, 7, 11, 14])
column_indices = np.array([0, 4, 0, 1, 1, 2, 3, 0, 2, 3, 4, 1, 3, 4])
values = np.array([10, -2, 3, 9, 7, 8, 7, 3, 8, 7, 5, 8, 9, 13])

csr_matrix((values, column_indices, row_ptr), shape=(len(row_ptr)-1, 5)).toarray()

array([[10,  0,  0,  0, -2],
       [ 3,  9,  0,  0,  0],
       [ 0,  7,  8,  7,  0],
       [ 3,  0,  8,  7,  5],
       [ 0,  8,  0,  9, 13]])

In [11]:

def tf_idf_transform(corpus: np.ndarray, token_dimenstion: dict) -> np.ndarray:
    """
    Transform the corpus into a sparse matrix 
    
    Parameters
    -----------
    corpus: np.ndarray, shape(n, )
    
    token_dimension:  a dict with token as key and basis-index as value
    
    Return
    ----------
    A normalized sparse matrix 
    """
    
    corpus = corpus.ravel()
    n = len(corpus)  # total number of documents
    rows = []
    columns = []
    tf_idf_values = []
    tf_val = []
    idf_val = []
    
    for idx, row in enumerate(corpus):
        # for each document
        word_freq = dict(Counter(row.split()))
        # total number of tokens in ONE document 
        d = len(row.split())
        # nested loop for each document 
        for word, freq in word_freq.items():
            # only retrieve those words with len >= 2
            if len(word) >= 2:
                # get the dimension as idx
                # if the key does not exist, dimension_idx = -1
                # see the above illustration 
                term_idx = token_dimenstion.get(word, -1)
                
                if term_idx != -1:
                    # first, store the index of the document
                    rows.append(idx)
                    # second, store the index of the word
                    columns.append(term_idx)
                    # calculate tf (term frequency) for one document
                    tf_d = freq/d
                    tf_val.append(tf_d)
                    # calculate IDF based on formula
                    # first we need to calculate df(t)
                    # term frequency in all the documents (corpus)
                    df_t = count_word(corpus, word)
                    idf_t = np.log((1+n)/(1+df_t)) + 1
                    idf_val.append(idf_t)
                    # calculate tf-idf
                    tf_idf = tf_d * idf_t
                    tf_idf_values.append(tf_idf)
                    
                    # create the sparse matrix 
                    sparse_matrix = csr_matrix(
                        (tf_idf_values, (rows, columns)),
                        shape = (len(corpus), len(token_dimenstion))
                    )
    return preprocessing.normalize(sparse_matrix)  # normalize the matrix

_Remark_: in the above matrix, we used:

- `csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])`
    - where data, row_ind and col_ind satisfy the relationship `a[row_ind[k], col_ind[k]] = data[k]`.

instead of 

- `csr_matrix((data, indices, indptr), [shape=(M, N)])`
    - is the standard CSR representation where the column indices for row i are 
    stored in `indices[indptr[i]:indptr[i+1]]` and their corresponding values 
    are stored in `data[indptr[i]:indptr[i+1]]`. If the shape parameter is not 
    supplied, the matrix dimensions are inferred from the index arrays.

In [12]:
# create the basis
token_basis_dict = construct_dimension(corpus_1)
corpus_csr = tf_idf_transform(corpus_1, token_basis_dict)

In [13]:
corpus_csr.toarray()  # or todense()

array([[0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524],
       [0.        , 0.6876236 , 0.        , 0.28108867, 0.        ,
        0.53864762, 0.28108867, 0.        , 0.28108867],
       [0.51184851, 0.        , 0.        , 0.26710379, 0.51184851,
        0.        , 0.26710379, 0.51184851, 0.26710379],
       [0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524]])

In [14]:
corpus_csr.shape

(4, 9)

For our sparse matrix, $A$ is of $4 \times 9$, which means 

- 4 documents
- 9 unique terms as basis 

In [22]:
# retrieve the terms value for the first documents
corpus_csr[0].T.todense()

matrix([[0.        ],
        [0.46979139],
        [0.58028582],
        [0.38408524],
        [0.        ],
        [0.        ],
        [0.38408524],
        [0.        ],
        [0.38408524]])

In [24]:
# convert the data into dataframe
tf_idf_table = pd.DataFrame(
    corpus_csr[0].T.todense(),
    index=token_basis_dict.keys(),
    columns=['tf-idf(document1)']
)
tf_idf_table.sort_values(by=['tf-idf(document1)'], ascending=False)

,tf-idf(document1)
first,0.580286
document,0.469791
is,0.384085
the,0.384085
this,0.384085
and,0.000000
one,0.000000
second,0.000000
third,0.000000


## Implement TF-IDF with sklearn 

In [3]:
tf_idf_vect = TfidfVectorizer()  # initialize the class 
tf_idf_vect.fit(corpus_1)
skl_csr_mat = tf_idf_vect.transform(corpus_1)

In [4]:
skl_csr_mat.toarray()

array([[0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524],
       [0.        , 0.6876236 , 0.        , 0.28108867, 0.        ,
        0.53864762, 0.28108867, 0.        , 0.28108867],
       [0.51184851, 0.        , 0.        , 0.26710379, 0.51184851,
        0.        , 0.26710379, 0.51184851, 0.26710379],
       [0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524]])

In [30]:
skl_csr_mat[0].T.toarray()

array([[0.        ],
       [0.46979139],
       [0.58028582],
       [0.38408524],
       [0.        ],
       [0.        ],
       [0.38408524],
       [0.        ],
       [0.38408524]])

In [33]:
tf_idf_table['sklearn-results'] = skl_csr_mat[0].T.toarray()

In [34]:
# the results should be exactly same 
tf_idf_table

,tf-idf(document1),sklearn-results
and,0.000000,0.000000
document,0.469791,0.469791
first,0.580286,0.580286
is,0.384085,0.384085
one,0.000000,0.000000
second,0.000000,0.000000
the,0.384085,0.384085
third,0.000000,0.000000
this,0.384085,0.384085


## Cosine Similarity

Now, we could like to use __cosine similarity__ to measure how similar of
_each document_ with its term frequency vector. Here is the formula

$$cs(x, y) = \frac{xy^T}{||x|| ||y||}$$

In [12]:
def pairwise_cosine_similarity(A: np.ndarray) -> np.ndarray:
    """
    Calculate cosine similarity for each row of matrix A based on the 
    cosine similarity formula in the pairwise fashion
    
    Parameters
    ------------
    A: np.ndarray shape (m, n) array
        m - number of documents;
        n - number of terms 
    
    Returns 
    ------------
    A matrix that contains the pa
    """
    # normalize the matrix first 
    A = A / np.linalg.norm(A, axis=1, keepdims=True)
    
    return A @ A.T

In [14]:
np.random.seed(567)
foo = np.random.rand(4, 5)
foo

array([[0.30478164, 0.95303297, 0.96470869, 0.34376214, 0.99388576],
       [0.30207403, 0.87623109, 0.70564267, 0.68115012, 0.54826595],
       [0.57399478, 0.930455  , 0.60177031, 0.54688225, 0.92238086],
       [0.15366693, 0.33696011, 0.7829969 , 0.61690568, 0.18440503]])

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(foo, foo)

array([[1.        , 0.94024168, 0.95788326, 0.80734086],
       [0.94024168, 1.        , 0.95618496, 0.90044363],
       [0.95788326, 0.95618496, 1.        , 0.77733634],
       [0.80734086, 0.90044363, 0.77733634, 1.        ]])

In [69]:
pairwise_cosine_similarity(foo)

array([[1.        , 0.94024168, 0.95788326, 0.80734086],
       [0.94024168, 1.        , 0.95618496, 0.90044363],
       [0.95788326, 0.95618496, 1.        , 0.77733634],
       [0.80734086, 0.90044363, 0.77733634, 1.        ]])

## Applications of TF-IDF

There are many applications of TF-IDF. According to wikipedia, a survey conducted 
in 2015 showed that 83% of text-based recommender systems in digital 
libraries use tf–idf.

Imagine you have a list of movies with `title`, `description`, you would like 
to get the similar movies based on `description`. Then you can use the 
cosine similarity based on the vector of TF-IDF. 

Now, will apply TF-IDF to the area of matching names of entities. Suppose we 
have a table A with a columns called `firm_names_a`, and we have a table B
with a column called `firm_names_b`. 

In [6]:
# read demf csv
demf = pd.read_csv('./data/demf.csv')
ict_def = pd.read_csv('./data/ict_de_firms.csv')

In [9]:
pd.DataFrame(
    {'firm_names_a': demf['name_internat'], 'firm_name_b': ict_def['name']}
).head()

,firm_names_a,firm_name_b
0,Dr. Upmeier Pestalozzistr. 36 KG,KUBLER-ALFERMI GMBH
1,Bruzkus Batek Architekten Partnerschaft,CELLUMATION GMBH
2,Kontakte-Umwelt e.V.,AIRBUS DEFENCE AND SPACE GMBH
3,JUNGE FREIHEIT Verlag GmbH & Co. KG,DOWAKSA DEUTSCHLAND GMBH
4,Witte Projektmanagement GmbH,HANSEATIC LLOYD SCHIFFAHRT GMBH & CO KG


Our task is to __match names__ between two different datasets and then merge
the database together. There are many methods to match names. The most
straightforward one is to use brute force to measure the distance between 
the strings. However, it takes lots of time when you have millions of names. 

One of the effective ways is to use TF-IDF vector and cosine similarity. 

<div>
<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
  "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<svg xmlns:xlink="http://www.w3.org/1999/xlink" width="460.8pt" height="105.282462pt" viewBox="0 0 460.8 105.282462" xmlns="http://www.w3.org/2000/svg" version="1.1">
 <metadata>
  <rdf:RDF xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:cc="http://creativecommons.org/ns#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
   <cc:Work>
    <dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
    <dc:date>2023-01-15T19:39:04.762365</dc:date>
    <dc:format>image/svg+xml</dc:format>
    <dc:creator>
     <cc:Agent>
      <dc:title>Matplotlib v3.5.2, https://matplotlib.org/</dc:title>
     </cc:Agent>
    </dc:creator>
   </cc:Work>
  </rdf:RDF>
 </metadata>
 <defs>
  <style type="text/css">*{stroke-linejoin: round; stroke-linecap: butt}</style>
 </defs>
 <g id="figure_1">
  <g id="patch_1">
   <path d="M 0 105.282462 
L 460.8 105.282462 
L 460.8 0 
L 0 0 
L 0 105.282462 
z
" style="fill: none"/>
  </g>
  <g id="axes_1">
   <g clip-path="url(#pb6db880d9f)">
    <image xlink:href="data:image/png;base64,
..." id="image81fe4d9aeb" transform="scale(1 -1)translate(0 -90.96)" x="7.2" y="-7.122462" width="446.4" height="90.96"/>
   </g>
  </g>
 </g>
 <defs>
  <clipPath id="pb6db880d9f">
   <rect x="7.2" y="7.2" width="446.4" height="90.882462"/>
  </clipPath>
 </defs>
</svg>

</div>

We have already shown that a very sparse matrix will be generated 
when we create the matrix of TF-IDF. We utilized the function `csr_matrix` from
SciPy. However, there are some issues when we use the original sparse matrix
production: 

- The original SciPy implementation has a lot of type checking, error handling etc. since it is a general-purpose function.
- If we use SciPy sparse matrix production, we have to sort the `top-n` values
in the end with `numpy.argpartition`. However, we could write our own `C++` function
and sort the `top-n` values in place. 
- The original SciPy implementation does two passes on the matrices 
(SciPy code is [here](https://github.com/scipy/scipy/blob/c43c0556b5c4565c903d7c16f9f28a9ddd5552d7/scipy/sparse/sparsetools/csr.h#L557)). The first one is to estimate how much memory to 
reserve for the result matrix, and the second is for the actual calculation. 
The first pass is not needed in our case, since we know that the maximum space 
needed for the result matrix is $M \times n$. The similarity scores 
(i.e. the result matrix entries) below a certain threshold can be ignored 
easily, so that they do not keep space in the memory and they are not 
involved in the partial sorting of the candidate list.

## Customize the sparse matrix product

Now,we will implement a customized `C++` function that does _compressed sparse row matrix
multiplication and select the top-n cosine similarities_ for the BIG DATA matching. If your
dataset is less than 1 million, the sparse matrix product function from SciPy 
should be fast enough. 

_Note_: when we calculate the cosine similarity above, we use the same matrix.
However we might have different matrices, meaning matrix $A$ is generated 
from corpus $A$ and matrix $B$ is generated from corpus $B$.

__Should we transform our data into one matrix or two matrices?__ In practice,
one should always transform our data into one matrix when the set of corpus
$A$ and corpus $B$ is not that big. However, when your dataset of corpus $A$ 
and $B$ are very big and they are very very similar, you should transform
them into two different matrices. 

To make our function work in the general case, we will assume the input of 
the function will be two different matrices:

- matrix $A$ of $M \times k$
    - where $M$ is the number of documents from corpus $A$
    - where $k$ is the number of features (or terms)
- matrix $B$ of $k \times N$
    - where $N$ is the number of documents from corpus B

<div>
<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
  "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<svg xmlns:xlink="http://www.w3.org/1999/xlink" width="460.8pt" height="152.26557pt" viewBox="0 0 460.8 152.26557" xmlns="http://www.w3.org/2000/svg" version="1.1">
 <metadata>
  <rdf:RDF xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:cc="http://creativecommons.org/ns#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
   <cc:Work>
    <dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
    <dc:date>2023-01-15T20:26:27.690084</dc:date>
    <dc:format>image/svg+xml</dc:format>
    <dc:creator>
     <cc:Agent>
      <dc:title>Matplotlib v3.5.2, https://matplotlib.org/</dc:title>
     </cc:Agent>
    </dc:creator>
   </cc:Work>
  </rdf:RDF>
 </metadata>
 <defs>
  <style type="text/css">*{stroke-linejoin: round; stroke-linecap: butt}</style>
 </defs>
 <g id="figure_1">
  <g id="patch_1">
   <path d="M 0 152.26557 
L 460.8 152.26557 
L 460.8 0 
L 0 0 
L 0 152.26557 
z
" style="fill: none"/>
  </g>
  <g id="axes_1">
   <g clip-path="url(#pc84b0fff57)">
    <image xlink:href="data:image/png;base64,
..." id="imaged5b99c3e09" transform="scale(1 -1)translate(0 -138)" x="7.2" y="-7.06557" width="446.4" height="138"/>
   </g>
  </g>
 </g>
 <defs>
  <clipPath id="pc84b0fff57">
   <rect x="7.2" y="7.2" width="446.4" height="137.86557"/>
  </clipPath>
 </defs>
</svg>

</div>

Since we are dealing with sparse matrices, we need to be careful when we 
are doing the dot production in `C++`. The general rule is to calcuate 

$$A[i, :] B = C_i, \quad for i \ \in [0, M-1]$$

which means we are iterating over rows of matrix $A$. The following
figure illustrates our algorithm implementation.

<div>
<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
  "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<svg xmlns:xlink="http://www.w3.org/1999/xlink" width="460.8pt" height="328.830848pt" viewBox="0 0 460.8 328.830848" xmlns="http://www.w3.org/2000/svg" version="1.1">
 <metadata>
  <rdf:RDF xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:cc="http://creativecommons.org/ns#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
   <cc:Work>
    <dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
    <dc:date>2023-01-16T11:02:39.543795</dc:date>
    <dc:format>image/svg+xml</dc:format>
    <dc:creator>
     <cc:Agent>
      <dc:title>Matplotlib v3.5.2, https://matplotlib.org/</dc:title>
     </cc:Agent>
    </dc:creator>
   </cc:Work>
  </rdf:RDF>
 </metadata>
 <defs>
  <style type="text/css">*{stroke-linejoin: round; stroke-linecap: butt}</style>
 </defs>
 <g id="figure_1">
  <g id="patch_1">
   <path d="M 0 328.830848 
L 460.8 328.830848 
L 460.8 0 
L 0 0 
L 0 328.830848 
z
" style="fill: none"/>
  </g>
  <g id="axes_1">
   <g clip-path="url(#p7d6e239701)">
    <image xlink:href="data:image/png;base64,
..." id="image8d3ee42696" transform="scale(1 -1)translate(0 -314.64)" x="7.2" y="-6.990848" width="446.4" height="314.64"/>
   </g>
  </g>
 </g>
 <defs>
  <clipPath id="p7d6e239701">
   <rect x="7.2" y="7.2" width="446.4" height="314.430848"/>
  </clipPath>
 </defs>
</svg>

</div>

Now, we will use our favoriate glue language `Python` to call our C++ code.
To do this, we need compile our C++ file via `Cpython`. We will use Cython to 
generate a `.pyx` file that contains a Python wrapper for C++ code. 

The `.pyx` file will contain Cython code that can be compiled into a Python 
extension module. This extension module can then be imported into Python and 
used to call the C++ code with a `setup.py` file. 

In [7]:
# import our function
import cos_sim_topn.cos_sim_topn as cs

In [8]:
# nowe we will write a python function to utilitze our c++ function
def awesome_cos_sim(A:np.ndarray, B:np.ndarray, topn:int, lower_bound=0):
    """
    Calculate the top n cosine similarities based on TF-IDF sparse matrices
    Parameters
    -----------
    A: CSR sparse matrix (M, k) made by scipy.sparse.csr_matrix 
    B: CSR sparse matrix (k, N) made by scipy.sparse.csr_matrix
    """
    A = A.tocsr()
    B = B.tocsr()
    M, ka = A.shape
    kb, N = B.shape
    assert ka == kb, \
         f"""
         The column dimension of A is {ka} does not equal to 
         the row dimension of B which is {kb};
         make sure you transposed your matrix B with feature x No. of Documents
         and matrix A and B should have the same dimension of features
         """
    idx_dtype = np.int32

    sparse_max_elements = M * topn 

    # initialize the result matrix C
    C_row_idx = np.zeros(M+1, dtype=idx_dtype)
    C_column_idx = np.zeros(sparse_max_elements, dtype=idx_dtype)
    # array type has to be double 
    C_data = np.zeros(sparse_max_elements, dtype=A.dtype)

    # check A.allclose([0]) or B.allclose([0])
    if len(A.indices) > 0 and len(A.data) > 0 and len(A.indptr) > 0 and \
        len(B.indices) > 0 and len(B.data) > 0 and len(B.indptr) > 0:
        cs.cos_sim_topn(
            M, N,
            np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            topn,
            lower_bound,
            C_row_idx, C_column_idx, C_data
        )

    # since column idx are created with grid of matrix, we keep
    # the shape as (M, N)
    return csr_matrix((C_data, C_column_idx, C_row_idx), shape=(M, N))

In [18]:
# let's test it, choose top 4
awesome_cos_sim(skl_csr_mat, skl_csr_mat.T, 4).toarray()

array([[1.        , 0.64692568, 0.30777187, 1.        ],
       [0.64692568, 1.        , 0.22523955, 0.64692568],
       [0.30777187, 0.22523955, 1.        , 0.30777187],
       [1.        , 0.64692568, 0.30777187, 1.        ]])

In [17]:
cosine_similarity(skl_csr_mat, skl_csr_mat)

array([[1.        , 0.64692568, 0.30777187, 1.        ],
       [0.64692568, 1.        , 0.22523955, 0.64692568],
       [0.30777187, 0.22523955, 1.        , 0.30777187],
       [1.        , 0.64692568, 0.30777187, 1.        ]])

In [19]:
# top 3, the smallest value becomes zero now 
awesome_cos_sim(skl_csr_mat, skl_csr_mat.T, 3).toarray()

array([[1.        , 0.64692568, 0.        , 1.        ],
       [0.64692568, 1.        , 0.        , 0.64692568],
       [0.30777187, 0.        , 1.        , 0.30777187],
       [1.        , 0.64692568, 0.        , 1.        ]])

## References

1. [TF-IDF : A visual explainer and Python Implementation on Presidential Inauguration Speeches](https://towardsdatascience.com/tf-idf-a-visual-explainer-and-python-implementation-on-presidential-inauguration-speeches-2a7671168550)

2. [ML Algorithms from Scratch with pure Python](https://www.kaggle.com/code/paulrohan2020/ml-algorithms-from-scratch-with-pure-python)

3. [Scipy Sparse Matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)

4. [Boosting Selection Of Most Similar Entities In Large Scale Datasets](https://www.sun-analytics.nl/posts/2017-07-26-boosting-selection-of-most-similar-entities-in-large-scale-datasets/)